# Surprising Pairs
**Find the pairs which appear to be the most non-linear in the pair-dataset.**

One complication: the set of notes for individual molecules is different from the notes of blend molecules. 

In [1]:
import json

# Load the dataset
with open("data/full.json") as f:
    dataset = json.load(f)
len(dataset), dataset[0]

(166814,
 {'mol1': 'CCCCC/C=C/C(=O)OC',
  'mol1_notes': ['violet',
   'sweet',
   'oily',
   'melon',
   'pear',
   'hairy',
   'costus',
   'fruity',
   'violet leaf',
   'waxy',
   'fresh',
   'green'],
  'mol2': 'CCCCCOC(=O)CCC',
  'mol2_notes': ['cherry',
   'sweet',
   'pineapple',
   'fruity',
   'banana',
   'tropical'],
  'blend_notes': ['animal', 'fruity', 'waxy']})

**Find the set of all notes and encode instances using one hot.**

In [2]:
import numpy as np

all_blend_notes = set()
all_single_notes = set()

for d in dataset:
    all_blend_notes.update(d["blend_notes"])
    all_single_notes.update(d["mol1_notes"])
    all_single_notes.update(d["mol2_notes"])

all_blend_notes = np.array(list(all_blend_notes))
all_single_notes = np.array(list(all_single_notes))
len(all_blend_notes), len(all_single_notes)

(109, 496)

In [3]:
def one_hot(notes,all_notes):
    encode = np.zeros(len(all_notes))
    for n in notes:
        encode[all_notes.index(n)] = 1
    return encode

one_hot(dataset[0]["blend_notes"], all_blend_notes.tolist())

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [4]:
from tqdm.notebook import tqdm

X = []
y = []

# Using sum for single notes b/c that is more expressive.
for d in tqdm(dataset):
    X.append(one_hot(d["mol1_notes"], all_single_notes.tolist()) + one_hot(d["mol2_notes"], all_single_notes.tolist()))
    y.append(one_hot(d["blend_notes"], all_blend_notes.tolist()))

X = np.stack(X)
y = np.stack(y)

X.shape, y.shape

  0%|          | 0/166814 [00:00<?, ?it/s]

((166814, 496), (166814, 109))

**Build logistic regression models to predict blend notes from single notes**

In [5]:
import sklearn
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier

models = []
for idx in tqdm(range(y.shape[1])):
    models.append(LogisticRegressionCV(class_weight="balanced",max_iter=1000).fit(X,y[:,idx]))
len(models), models[0]

  0%|          | 0/109 [00:00<?, ?it/s]

/Users/laurasisson/miniconda3/envs/rdenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/laurasisson/miniconda3/envs/rdenv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


(109, LogisticRegressionCV(class_weight='balanced', max_iter=1000))

**Calculate predictions and probability of label**

In [6]:
prob = np.stack([model.predict_proba(X) for model in tqdm(models)],axis=-1)
# Get probability for positive label
prob = prob[:,1,:]

pred = np.stack([model.predict(X) for model in tqdm(models)],axis=-1)
pred.shape, prob.shape

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

((166814, 109), (166814, 109))

**Calculate log-likelihood**

In [7]:
from scipy import sparse
# ---------- choose a loss / residual ----------
eps = 1e-12                               # numerical safety

# y is sparse
y_dense   = y.toarray()
logloss_each = -np.sum(
        y_dense * np.log(prob + eps) +
        (1 - y_dense) * np.log(1 - prob + eps),
        axis=1)                           # shape (n_samples,)
logloss_each.shape

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [ ]:
import matplotlib.pyplot as plt

# plt.hist(np.exp(-logloss_each))
plt.hist(logloss_each)
plt.title("Distribution of Negative Log-Likelihood\nby Logistic-Regression Model")

plt.yscale("log")

plt.xlabel("Negative Log-likelihood of Ground-Truth Labels")
plt.ylabel("Frequency (log-scale)")

plt.show()

**Sort by least-likely and generate output**

In [ ]:
import copy

sorted_indices = np.argsort(-logloss_each)
top_k = 100
output_dataset = []

for idx in sorted_indices[:top_k]:
    output = copy.deepcopy(dataset[idx])
    
    # Put predictions into the output entry
    output["pred_blend"] = all_blend_notes[np.where(pred[idx]==1)].tolist()
    output["log-likelihood"] = logloss_each[idx]
    
    # Sort both lists for clarity
    output["blend_notes"] = sorted(output["blend_notes"])
    output["pred_blend"] = sorted(output["pred_blend"])
    
    output_dataset.append(output)

len(output_dataset), output_dataset[0]

In [ ]:
with open("Output/unlikely_pairs.json","w") as f:
    json.dump(output_dataset,f)